# 简介

在这个Notebook里面，我将尝试利用现有技术实现一个基于Transformer的入侵检测系统并尽可能对其进行改进。首先，我将描述本架构的各个组成部分，在阐明其原理后对模型进行训练与评估，并在最后在相同问题下将该架构与其他架构进行对比。

以下是为训练此模型而导入的包：

In [1]:
!pip install einops
# !pip install numpy
# !pip install pandas
# !pip install scikit-learn
# !pip install imbalanced-learn
# !pip install rtdl_num_embeddings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 736.4 kB/s eta 0:00:00


以下代码用于挂载google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 结构

## 嵌入(Embedding)

在该部分里，数值信息将被转换为可被Transformer识别的特征值并提供给后面的结构进行学习。

$$ReLU(Linear(Periodic(x_i)))$$

In [3]:
from torch import nn
import torch
from einops import rearrange

class Embedder(nn.Module):
    def __init__(self, dim, num_numerical_types=10):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(num_numerical_types, dim))
        self.biases = nn.Parameter(torch.randn(num_numerical_types, dim))

    def forward(self, x):
        x = rearrange(x, 'b n -> b n 1')
        return x * self.weights + self.biases

## 位置编码(Positional Encoding)

由于嵌入产生的信息并不带有位置信息，故需要在学习前将位置信息写入训练数据中。

$$PE(pos, 2i) = \sin(pos/1000^{2i/d\_model})$$
$$PE(pos, 2i+1) = \cos(pos/1000^{2i/d\_model})$$

In [4]:
import torch
from torch import nn
from torch.autograd import Variable
from math import cos, sin, sqrt

class Positional_Encoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 10):
        super().__init__()
        self.d_model = d_model

        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0,d_model,2):
                pe[pos,i] = sin(pos / (1000**(i/d_model)))
                pe[pos,i+1] = cos(pos / (1000**(i/d_model)))

        pe = pe.unsqueeze(0)
        self.register_buffer("pe",pe)

    def forward(self, x):
        x = x * sqrt(self.d_model)
        seq_len = x.size(1)
        y = Variable(self.pe[:,:seq_len],requires_grad=False).cuda()
        x = x + y
        return x

## 注意力机制(Attention)

本部分用于对输入向量进行处理以获取输入与输出间的对应关系。具体计算过程如下：

$$Attention(Q,K,V) = softmax\left(M+\frac{QK^T}{\sqrt{d_k}}\right)$$

In [19]:
import torch
from torch import nn
from torch.nn import functional as F
import math

def attention(q, k, v, d_k, mask=None, dropout=None):
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    if mask is not None:
      shape = scores.shape
      mask = get_mask(shape[0], 8, 10)
      scores = scores.masked_fill(mask == 0, -1e9)
    scores = F.softmax(scores, dim=-1)

    if dropout is not None:
        scores = dropout(scores)

    output = torch.matmul(scores, v)
    return output

class Multi_Head_Attention(nn.Module):
    def __init__(self, heads, d_model, dropout):
        super().__init__()
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):

        bs = q.size(0)

        # perform linear operation and split into h heads
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)

        # transpose to get dimensions bs * h * sl * d_model
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)

        scores = attention(q, k, v, self.d_k, mask, self.dropout)

        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous().view(bs, -1, self.d_model)

        output = self.out(concat)
        return output


## 掩蔽(Masks)

该部分用于对输入解码器(Decoder)的向量进行掩蔽来使得解码器能够根据上下文来推算结果。

In [6]:
import numpy as np
import torch
from torch.autograd import Variable

def get_mask(batch_size, heads, seq_size):
    mask_prob = 0.2
    mask = torch.rand((batch_size, heads, seq_size, seq_size)) > mask_prob
    return mask.cuda()

## 前馈网络(Feed-Forward Network)

该部分主要用于记忆注意力机制计算所产生的的关系。具体原理如下：

$$FFN(x) = \max(0,xW_1 + b_1)W_2 + b_2$$

In [7]:
from torch import nn
from torch.nn import functional as F

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=1024, dropout = 0.1):
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

## 层归一化(Layer Norm)

主要用于使不同范围的数据归一化到[0, 1]区间内，方便模型进行处理，具体原理如下：

$$LN(x) = \frac{x - \mu}{\delta}\cdot \alpha + \beta$$

In [8]:
from torch import nn
import torch

class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()

        self.size = d_model

        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

## 编码器(Encoder)

整个编码器由数个编码器嵌入器、数个编码器层与层归一化层组成。其中，一个编码器层由层归一化、注意力机制与前馈神经网络层组成，数据在经过这些处理后由dropout方法进行输出。

In [9]:
from torch import nn
import copy

def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class Encoder_Layer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = Multi_Head_Attention(heads, d_model, dropout)
        self.feedf = FeedForward(d_model).cuda()
        self.dropout_1 = nn.Dropout(dropout).cuda()
        self.dropout_2 = nn.Dropout(dropout).cuda()

    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.feedf(x2))
        return x

class Encoder(nn.Module):
    def __init__(self, d_model, N, heads, dropout = 0.1):
        super().__init__()
        self.N = N
        self.embed = Embedder(d_model)
        self.pe = Positional_Encoder(d_model)
        self.layers = get_clones(Encoder_Layer(d_model, heads, dropout), N)
        self.norm = Norm(d_model)

    def forward(self, src, mask = None):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x,mask)
        return self.norm(x)

## 解码器

In [10]:
import torch
from torch import nn
import copy

def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class Decoder_Layer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        self.attn = Multi_Head_Attention(heads, d_model, dropout)
        self.msk_attn = Multi_Head_Attention(heads, d_model, dropout)
        self.feedf = FeedForward(d_model).cuda()
        self.dropout_1 = nn.Dropout(dropout).cuda()
        self.dropout_2 = nn.Dropout(dropout).cuda()
        self.dropout_3 = nn.Dropout(dropout).cuda()

    def forward(self, x, e_outputs, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.msk_attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn(x2,e_outputs,e_outputs,mask=None))
        x2 = self.norm_3(x)
        x = x +self.dropout_3(self.feedf(x2))
        return x

class Decoder(nn.Module):
    def __init__(self, d_model, N, heads, dropout = 0.1):
        super().__init__()
        self.N = N
        self.embed = Embedder(d_model)
        self.pe = Positional_Encoder(d_model,1)
        self.layers = get_clones(Decoder_Layer(d_model, heads, dropout), N)
        self.norm = Norm(d_model)

    def forward(self, trg, e_outputs, mask = None):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x,e_outputs,mask)
        return self.norm(x)


## 整体架构定义

In [11]:
import torch
from torch import nn

class Transformer(nn.Module):
    def __init__(self, trg_vocab, d_model, N, heads, dropout = 0.1):
        super().__init__()
        self.encoder = Encoder(d_model, N , heads, dropout)
        self.decoder = Decoder(d_model, N , heads, dropout)
        self.out = nn.Linear(10*d_model, trg_vocab)

    def forward(self, src, trg_mask=None):
        e_outputs = self.encoder(src, None)
        d_output = self.decoder(src, e_outputs, trg_mask)
        d_intermediate = d_output.view(d_output.size(0), -1)
        output = self.out(d_intermediate)
        output = torch.softmax(output,dim=1)
        return output

# 数据集与数据处理

## NF-CEC-CIC-IDS2018-v2
下载数据集:https://rdm.uq.edu.au/files/ce5161d0-ef9c-11ed-827d-e762de186848

昆士兰大学Sarhan团队利用了CSE-CIC-IDS2018数据集的原始pcap文件生成了一个基于NetFlow的数据集，称为NF-CSE-CIC-IDS2018-v2。总的流量数量为18,893,708，其中2,258,141（11.95%）是攻击样本，16,635,567（88.05%）是良性样本。

## 数据处理
本部分用于对数据进行预处理。由于本数据集的特殊性——正常流量远多于异常攻击流量，因此需要对数据进行下采样与SMOTE处理，前者用于限制数量较多的样本的数量，后者用于对数量较少的样本进行插值处理以增加其数量。

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.neighbors import NearestNeighbors
from imblearn.over_sampling import KMeansSMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from itertools import chain
import glob
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os

label_mapping = {}
directory_path = 'drive/MyDrive/Colab Notebooks/SIMARGL2021/data/'


class CICIDSDataset(Dataset):
    def __init__(self, data):
        self.features = data[:, :-3]  # Feature Columns
        self.at_type = data[:, -3]  # Attack Type Column
        self.labels = data[:, -2:]  # 1 Hot Encoded Label

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        feature = torch.from_numpy(self.features[idx]).float()
        label = self.labels[idx]
        at_type = self.at_type[idx]

        return feature, label, at_type


def data_process(data, label_mapping):
    # Undersampling & SMOTE
    max_class_size = 100000  # Size of all Classes for Undersampling
    class_counts = data['LABEL'].value_counts()
    classes_to_undersample = class_counts[class_counts > max_class_size].index

    under_sampler = RandomUnderSampler(sampling_strategy={
        label: 7 * max_class_size if label == "Normal flow" else max_class_size if label in classes_to_undersample else
        class_counts[label] for label in np.unique(data['LABEL'])
    }, random_state=42)
    nn_estimator = NearestNeighbors(n_neighbors=5, n_jobs=-1)
    KSMOTE = KMeansSMOTE(sampling_strategy={
        label: 7 * max_class_size if label == "Normal flow" else max_class_size for label in np.unique(data['LABEL'])
    }, k_neighbors=nn_estimator, cluster_balance_threshold=0.001, random_state=42)
    scaler = MinMaxScaler()

    features = data.drop(columns=['LABEL'])
    labels = data['LABEL']
    sampled_features, sampled_labels = under_sampler.fit_resample(features, labels)
    balanced_features, balanced_labels = KSMOTE.fit_resample(sampled_features, sampled_labels)
    scaled_data = scaler.fit_transform(balanced_features)

    data = pd.DataFrame(data=scaled_data, columns=features.columns)
    data = pd.concat([data, balanced_labels.map(label_mapping)], axis=1)

    # 1 Hot Encoding
    data['Attack_Label'] = data['LABEL'].apply(lambda x: "Attack" if x != label_mapping["Normal flow"] else "Normal flow")
    encoded_labels = pd.get_dummies(data['Attack_Label'], prefix='', prefix_sep='')
    data = pd.concat([data, encoded_labels], axis=1)
    data.drop('Attack_Label', axis=1, inplace=True)

    label_counts = data['LABEL'].value_counts()
    print(label_counts)
    return data


def load_data():
    if os.path.exists(directory_path + "preprocessed/data1.csv.gz"):
        print("Loading Preprocessed Data")
        data = pd.read_csv(directory_path + "preprocessed/data.csv.gz", compression='gzip')

    else:
        chunk_size = 100000
        csv_files = glob.glob(directory_path + '*.csv.gz')

        chunks = []
        pro = []
        atk = []
        for file in csv_files:
            pro_chunk = pd.read_csv(file, usecols=['L7_PROTO_NAME'])
            atk_chunk = pd.read_csv(file, usecols=['LABEL'])
            dataframe = pd.read_csv(file, chunksize=chunk_size, compression='gzip', low_memory=False)
            chunks.append(dataframe)
            pro.append(pro_chunk)
            atk.append(atk_chunk)
        atk = pd.concat(atk, ignore_index=True)
        pro = pd.concat(pro, ignore_index=True)

        pro_encoder = LabelEncoder()
        atk_encoder = LabelEncoder()
        atk_encoder.fit_transform(atk.values.ravel())
        pro_encoder.fit_transform(pro.values.ravel())
        pro_mapping = dict(enumerate(pro_encoder.classes_))
        atk_mapping = dict(enumerate(atk_encoder.classes_))
        pro_mapping = {value: key for key, value in pro_mapping.items()}
        atk_mapping = {value: key for key, value in atk_mapping.items()}
        del pro_encoder
        del atk_encoder

        chunks = chain(*chunks)

        max_float64 = np.finfo(np.float64).max

        data = []
        mem = 0
        cnt = 1
        for chunk in chunks:
            chunk.drop(chunk.columns[[17, 18, 29]], axis=1, inplace=True)
            chunk['SRC_TO_DST_SECOND_BYTES'] = chunk['SRC_TO_DST_SECOND_BYTES'].apply(lambda x: hash(x)) % 1000000
            chunk['DST_TO_SRC_SECOND_BYTES'] = chunk['DST_TO_SRC_SECOND_BYTES'].apply(lambda x: hash(x)) % 1000000
            chunk['L7_PROTO_NAME'] = chunk['L7_PROTO_NAME'].map(pro_mapping)

            chunk.columns = chunk.columns.str.strip()
            chunk = chunk.dropna()

            feature_chunk = chunk.drop(columns=['LABEL'])
            numeric_columns = feature_chunk.select_dtypes(include=[np.number]).columns
            feature_chunk[numeric_columns] = np.minimum(feature_chunk[numeric_columns], max_float64)
            labels_chunk = chunk['LABEL']
            chunk = pd.concat([feature_chunk, labels_chunk], axis=1)
            mem += (chunk.memory_usage(deep=True)).sum() / (1024 * 1024 * 1024)
            data.append(chunk)
            # 当读取的数据集大于等于一定大小时，只对该数据块进行处理
            if mem >= 9.5:
                data = pd.concat(data, ignore_index=True)
                mem = 0

                data = data_process(data, atk_mapping)

                # print(data['LABEL'].value_counts())
                if not os.path.exists(directory_path + 'preprocessed'):
                    os.makedirs(directory_path + 'preprocessed')
                data.to_csv(directory_path + 'preprocessed/data' + str(cnt) + '.csv.gz', index=False,
                            compression='gzip')
                cnt += 1
                print("saved")
                data = []
        else:
            data = pd.concat(data, ignore_index=True)
            data = data_process(data, atk_mapping)

            print(data['Attack'].value_counts())
            if not os.path.exists(directory_path + 'preprocessed'):
                os.makedirs(directory_path + 'preprocessed')
            data.to_csv(directory_path + 'preprocessed/data' + str(cnt) + '.csv.gz', index=False, compression='gzip')
            cnt += 1
            print("saved")

        data = glob.glob(directory_path + 'preprocessed/' + '*.gz')
        dataframes = []
        for file in data:
            dataframe = pd.read_csv(file, compression='gzip')
            dataframes.append(dataframe)
        data = pd.concat(dataframes, ignore_index=True)
        del dataframes

        features = data.drop(columns=['LABEL', 'Attack', 'Normal flow'])
        labels = data['LABEL']

        # Undersampling & SMOTE
        max_class_size = 100000  # Size of all Classes for Undersampling
        class_counts = data['LABEL'].value_counts()
        classes_to_undersample = class_counts[class_counts > max_class_size].index
        under_sampler = NearMiss(sampling_strategy={
          label: 7 * max_class_size if label == "Normal flow" else max_class_size if label in classes_to_undersample else
          class_counts[label] for label in np.unique(data['LABEL'])
        })
        sampled_features, sampled_labels = under_sampler.fit_resample(features, labels)
        sampled_labels = pd.concat([sampled_labels, data['Attack'], data['Normal flow']], axis=1)

        # PCA features extraction
        print('Using PCA...')
        scaler = StandardScaler()
        features = np.nan_to_num(scaler.fit_transform(sampled_features))
        pca = PCA(n_components=10, svd_solver='randomized')
        features = pd.DataFrame(pca.fit_transform(features))
        data = pd.concat([features, sampled_labels], axis=1)

        data.iloc[:, :10] = data.iloc[:, :10].astype('float64')
        data.iloc[:, -3:] = data.iloc[:, -3:].astype('int')

        data.to_csv(directory_path + 'preprocessed/data.csv.gz', index=False, compression='gzip')

    train_data, val_data = train_test_split(data, test_size=0.25, random_state=42)
    train_data, val_data = train_data.values, val_data.values

    return train_data, val_data


def get_data_loader(data, batch_size):
    cicids_dataset = CICIDSDataset(data)

    return DataLoader(cicids_dataset, batch_size=batch_size, shuffle=True)

# 模型训练

定义训练与评估函数

In [20]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import time
import os
import torch
from torch.nn import functional as F
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, r2_score, \
    mean_squared_error, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay


def plot_metrics(losses, accuracies):
    # 绘制损失和准确率曲线
    epochs = range(1, len(losses) + 1)
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, losses, 'r', label='Training Loss')
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracies, 'b', label='Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()
    plt.show()


def eval_model(model, loader):
    model.cuda()
    model.eval()
    losses = []
    correct = 0
    predictions = []
    targets = []

    with torch.no_grad():
        for data, target, _ in loader:
            data, target = data.to("cuda"), target.to("cuda")
            output = model(data)
            loss = F.cross_entropy(output, target).item()
            losses.append(loss)
            pred = torch.argmax(output, dim=1)
            correct += torch.eq(torch.argmax(output, dim=1),torch.argmax(target, dim=1)).cpu().sum().item()
            predictions.extend(pred.cpu().numpy())
            targets.extend(target.cpu().numpy())

    eval_loss = np.mean(losses)
    eval_acc = correct / len(loader.dataset)
    targets = torch.tensor(targets)

    # 计算精确率、召回率、F1 分数
    precision = precision_score(torch.argmax(targets, dim=1), predictions, average='macro')
    recall = recall_score(torch.argmax(targets, dim=1), predictions, average='macro')
    f1 = f1_score(torch.argmax(targets, dim=1), predictions, average='macro')

    # 计算 ROC 曲线和 AUC
    try:
        auc = roc_auc_score(torch.argmax(targets, dim=1), predictions, average='weighted')
    except ValueError:
        auc = None

    # 计算混淆矩阵
    conf_matrix = confusion_matrix(torch.argmax(targets, dim=1), predictions)

    # 计算 R-squared
    r_squared = r2_score(torch.argmax(targets, dim=1), predictions)

    # 计算均方误差
    mse = mean_squared_error(torch.argmax(targets, dim=1), predictions)

    # 计算对数损失
    try:
        logloss = log_loss(torch.argmax(targets, dim=1), predictions)
    except ValueError:
        logloss = None

    print("Loss:", eval_loss, "Accuracy:", eval_acc)
    print("Precision:", precision, "Recall:", recall, "F1 Score:", f1)
    print("AUC:", auc)
    print("Confusion Matrix:")
    print(conf_matrix)
    print("R-squared:", r_squared)
    print("Mean Squared Error:", mse)
    print("Log Loss:", logloss)

    return eval_loss, eval_acc, precision, recall, f1, auc, r_squared, mse, logloss, conf_matrix


def train_model(model, opt, epochs, data, eval_data, path, print_every=100):
    model.cuda()

    pretrained_path = "drive/MyDrive/Colab Notebooks/SIMARGL2021/pretrained"
    top_acc = 0.

    if os.path.exists(pretrained_path + "/" + path):
        print("Loading Pretrained Model")
        state = torch.load(pretrained_path + "/" + path)
        model.load_state_dict(state["model_state_dict"])
        start_epoch = state["epoch"] + 1
        losses = state["ep_loss"]
        accs = state["ep_acc"]
        top_acc = max(accs)
    else:
        start_epoch = 0
        losses, accs = [], []
        try:
            os.mkdir(pretrained_path)
        except OSError as error:
            pass

    start = time.time()
    temp = start

    for epoch in range(start_epoch, epochs):
        model.train()
        total_loss = 0
        for i, batch in enumerate(data):
            src, trg, _ = batch
            src, trg = src.cuda(), trg.cuda()

            if isinstance(model, Transformer):
                trg_mask = get_mask(128, 8, 10)
            else:
                trg_mask = None

            preds = model(src, trg_mask)
            opt.zero_grad()
            loss = F.cross_entropy(preds, trg)
            loss.backward()
            opt.step()

            total_loss += loss.data
            if (i + 1) % print_every == 0:
                loss_avg = total_loss / print_every
                print("time = %dm, epoch %d, iter = %d, loss = %.3f, \
                %ds per %d iters" % ((time.time() - start) // 60,
                                     epoch + 1, i + 1, loss_avg, time.time() - temp,
                                     print_every))
                total_loss = 0
                temp = time.time()
        eval_loss, eval_acc, precision, recall, f1, auc, r_squared, mse, logloss, conf_matrix = eval_model(model, eval_data)
        losses.append(eval_loss)
        accs.append(eval_acc)
        plot_metrics(losses, accs)

        if eval_acc > top_acc:
            top_state = {
                "model_state_dict": model.state_dict(),
                "epoch": epoch
            }
            torch.save(top_state, pretrained_path + "/max_" + path)
        state = {
            'model_state_dict': model.state_dict(),
            'epoch': epoch,
            'ep_loss': losses,
            'ep_acc': accs,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'conf_matrix': conf_matrix
        }
        torch.save(state, pretrained_path + "/" + path)

设置参数并训练模型

In [21]:
import torch
from torch import nn
from torchsummary import summary

learning_rate = 5e-4
batch_size = 128
epochs = 30
dropout_rate = 0.5
d_model = 32
heads = 8
N = 6
trg_vocab = 2

train_data, val_data = load_data()

train_loader = get_data_loader(train_data, batch_size)
val_loader = get_data_loader(val_data, batch_size)

model = Transformer(trg_vocab, d_model, N, heads, dropout_rate)
save_path = "pretrained.pt"

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
model = model.to("cuda:0")
summary(model, (10, ))

optim = torch.optim.RAdam(model.parameters(), lr=learning_rate)

train_model(model, optim, epochs, train_loader, val_loader, save_path)

Loading Preprocessed Data
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
          Embedder-1               [-1, 10, 32]               0
Positional_Encoder-2               [-1, 10, 32]               0
              Norm-3               [-1, 10, 32]              32
            Linear-4               [-1, 10, 32]           1,056
            Linear-5               [-1, 10, 32]           1,056
            Linear-6               [-1, 10, 32]           1,056
           Dropout-7            [-1, 8, 10, 10]               0
            Linear-8               [-1, 10, 32]           1,056
Multi_Head_Attention-9               [-1, 10, 32]               0
          Dropout-10               [-1, 10, 32]               0
             Norm-11               [-1, 10, 32]              32
           Linear-12             [-1, 10, 1024]          33,792
          Dropout-13             [-1, 10, 1024]               0
           

KeyboardInterrupt: 